In [1]:
# import libraries
import numpy as np
import sys
import psi4
from helper_PFCI import PFHamiltonianGenerator
np.set_printoptions(threshold=sys.maxsize)
#psi4.core.set_output_file('output.dat', False)
import time
from scipy.optimize import curve_fit
from scipy import interpolate
from matplotlib import pyplot as plt
from numpy.polynomial import Chebyshev


In [ ]:
min_length = 1.0  # Minimum bond length
max_length = 2.2  # Maximum bond length

# Define the equilibrium bond length
equilibrium_length = 1.5500000000000003  # Adjust as needed

# Specify the number of data points you want near the equilibrium bond length
num_points_near_equilibrium = 25

# Specify the number of data points in the remaining range
num_points_remaining = 25

# Create a non-uniform grid
grid_near_equilibrium = np.linspace(equilibrium_length - 0.5, equilibrium_length + 0.5, num_points_near_equilibrium)
grid_remaining = np.linspace(min_length, max_length, num_points_remaining)

# Combine the grids to create the non-uniform grid
#r_array = np.sort(np.concatenate((grid_near_equilibrium, grid_remaining)))
#print(r_array)

In [2]:
def chebyshev_nodes(n):
    return 0.5 * (1.0 - np.cos(np.pi * (2.0 * np.arange(1, n + 1) - 1) / (2.0 * n)))

def create_chebyshev_grid(start, end, num_points):
    nodes = chebyshev_nodes(num_points)
    scaled_nodes = start + nodes * (end - start)
    return scaled_nodes

r_array = create_chebyshev_grid(1.0, 2.2, 50)

print(r_array)


[1.00029606 1.00266282 1.007387   1.01444994 1.02382379 1.03547154
 1.04934722 1.06539609 1.08355478 1.10375166 1.12590699 1.14993336
 1.17573593 1.20321288 1.23225577 1.26274997 1.29457515 1.3276057
 1.36171127 1.39675725 1.43260534 1.46911406 1.50613932 1.54353501
 1.58115354 1.61884646 1.65646499 1.69386068 1.73088594 1.76739466
 1.80324275 1.83828873 1.8723943  1.90542485 1.93725003 1.96774423
 1.99678712 2.02426407 2.05006664 2.07409301 2.09624834 2.11644522
 2.13460391 2.15065278 2.16452846 2.17617621 2.18555006 2.192613
 2.19733718 2.19970394]


In [3]:
# setup basic arguments for qed-ci calculation

# z-matrix for CO
mol_str_CO = """
C
O 1 1.13
symmetry c1
"""

# options for Psi4
options_dict = {
    "basis": "sto-3g",
    "scf_type": "pk",
    "e_convergence": 1e-10,
    "d_convergence": 1e-10,
}

# options for the PFHamiltonian Generator class - include cavity effects
cavity_dict = {
    'omega_value' : 0.12086,
    'lambda_vector' : np.array([0, 0, 0.05]),
    'ci_level' : 'fci',   
    'full_diagonalization' : True,
    'number_of_photons' : 1, #<== this is a minimal photon basis, should explore increasing this 
}

# options for PFHamiltonian Generator class - exclude cavity effects
cavity_free_dict = {
    'omega_value' : 0.0,
    'lambda_vector' : np.array([0, 0, 0.0]),
    'ci_level' : 'fci',   
    'full_diagonalization' : True,
    'number_of_photons' : 0, 
}



In [ ]:
# template for the z-matrix so we can scan through bond lengths
mol_tmpl_CO = """
C
O 1 **R**
symmetry c1
"""


# number of bondlengths in the scan
N_R = 50

# number of electronic states to save
N_el = 10

#r_array = np.linspace(0.2, 2.2, N_R)


# array for energies for CO outside cavity
cavity_free_E_array_CO = np.zeros((N_R, N_el))

# array  for energies inside the cavity
cavity_E_array_CO = np.zeros((N_R, N_el))

r_idx_CO = 0
for r in r_array:
    mol_str_CO = mol_tmpl_CO.replace("**R**", str(r))
    mol_CO = psi4.geometry(mol_str_CO)
    print(mol_str_CO)
    
    # call psi4 RHF
    scf_e, wfn_CO = psi4.energy('SCF/sto-3g', return_wfn=True)
    
    # call psi4 FCI
    fci_energy_CO = psi4.energy('fci/sto-3g',ref_wfn=wfn_CO)
    
    # CO no cavity
    CO = PFHamiltonianGenerator(mol_str_CO, options_dict, cavity_free_dict)

    # check to make sure PFHamiltonian cavity-free FCI ground-state agrees with psi4
    assert np.isclose(CO.CIeigs[0], fci_energy_CO)
    
    # CO with cavity
    CO_cav = PFHamiltonianGenerator(mol_str_CO, options_dict, cavity_dict)
    cavity_free_E_array_CO[r_idx_CO,:] = CO.CIeigs[:N_el]
    cavity_E_array_CO[r_idx_CO,:] = CO_cav.CIeigs[:N_el]
    r_idx_CO += 1
    
    
    


C
O 1 1.0002960637805611
symmetry c1


Scratch directory: /tmp/
   => Libint2 <=

    Primary   basis highest AM E, G, H:  5, 4, 3
    Auxiliary basis highest AM E, G, H:  6, 5, 4
    Onebody   basis highest AM E, G, H:  6, 5, 4
    Solid Harmonics ordering:            gaussian

*** tstart() called on CHEMYM0V4HTALT
*** at Wed Nov 15 16:02:43 2023

   => Loading Basis Set <=

    Name: STO-3G
    Role: ORBITAL
    Keyword: BASIS
    atoms 1 entry C          line    61 file /Users/ptolley1/miniforge3/envs/work/share/psi4/basis/sto-3g.gbs 
    atoms 2 entry O          line    81 file /Users/ptolley1/miniforge3/envs/work/share/psi4/basis/sto-3g.gbs 


         ---------------------------------------------------------
                                   SCF
               by Justin Turney, Rob Parrish, Andy Simmonett
                          and Daniel G. A. Smith
                              RHF Reference
                        1 Threads,    500 MiB Core
         ----------------------

 Completed constant offset matrix in 1.7359108924865723 seconds
 Completed Hamiltonian build in 26855.454346895218 seconds

Scratch directory: /tmp/
   => Libint2 <=

    Primary   basis highest AM E, G, H:  5, 4, 3
    Auxiliary basis highest AM E, G, H:  6, 5, 4
    Onebody   basis highest AM E, G, H:  6, 5, 4
    Solid Harmonics ordering:            gaussian

*** tstart() called on CHEMYM0V4HTALT
*** at Thu Nov 16 02:19:48 2023

   => Loading Basis Set <=

    Name: STO-3G
    Role: ORBITAL
    Keyword: BASIS
    atoms 1 entry C          line    61 file /Users/ptolley1/miniforge3/envs/work/share/psi4/basis/sto-3g.gbs 
    atoms 2 entry O          line    81 file /Users/ptolley1/miniforge3/envs/work/share/psi4/basis/sto-3g.gbs 


         ---------------------------------------------------------
                                   SCF
               by Justin Turney, Rob Parrish, Andy Simmonett
                          and Daniel G. A. Smith
                              RHF Referenc

 Completed determinant list in 0.10018467903137207 seconds 
 Completed constant offset matrix in 1.2587132453918457 seconds
 Completed Hamiltonian build in 32105.462479114532 seconds


In [ ]:
# z-matrix for CO
mol_str_CO = """
Li
H 1 1.8
symmetry c1
"""

# options dictionary for the psi4 calculation
options_dict0 = {'basis': 'sto-3g',
               'save_jk': True, 
               'scf_type': 'pk',
               'e_convergence' : 1e-8,
               'd_convergence' : 1e-7
               }

# let's run a psi4 geometry optimization to get the optimal geometry

# set the options
psi4.set_options(options_dict0)

# first set the coordinates in init_string as the molecule's geometry 
mol0_CO = psi4.geometry(mol_str_CO)

# now capture the psi4 geometry data with the guess coordinates
init_geometry_string_CO = psi4.core.Molecule.create_psi4_string_from_molecule(mol0_CO)

# run the geometry optimization
psi4.optimize("fci")

guess_geometry_CO = mol0_CO.geometry()
print(guess_geometry_CO)

guess_string_CO = psi4.core.Molecule.create_psi4_string_from_molecule(mol0_CO)


In [ ]:
# options dictionary for the psi4 calculation
options_dict = {'basis': 'sto-3g',
               'save_jk': True, 
               'scf_type': 'pk',
               'e_convergence' : 1e-8,
               'd_convergence' : 1e-7
               }

# set the options
psi4.set_options(options_dict)

# first set the coordinates in guess_string as the molecule's geometry 
mol_CO = psi4.geometry(guess_string_CO)

# now capture the psi4 geometry data with the guess coordinates
guess_geometry_string_CO = psi4.core.Molecule.create_psi4_string_from_molecule(mol_CO)

# run the geometry optimization
psi4.optimize("fci")

opt_geometry = mol_CO.geometry()

# capture the psi4 geometry data from the optimization
opt_geometry_string_CO = psi4.core.Molecule.create_psi4_string_from_molecule(mol_CO)

# run excited-state calculations - get first 5 excited states
n_states = 1

# run psi4 TDSCF at same level of theory as geometry optimization
psi4_rhf_e, wfn_CO = psi4.frequency("fci/sto-3g", return_wfn=True, molecule=mol_CO,maxiter=50)



In [ ]:
# Morse parameters
De_au = 0.4125169968027636
beta_au = 1.3755432414187434
r_eq_au = 2.132177987349192
N_points = 20 # this will depend on the order of your Chebyshev polynomial
# replace this linear spaced grid with a Chebyshev grid betewen 0.75 * r_eq and 2.5 * r_eq
r = np.linspace(0.8 * r_eq_au, 1.2 * r_eq_au, 20)
# evaluate the Morse potential on that Chebyshev grid
V_r = De_au * (1 - np.exp(-beta_au * (r - r_eq_au))) ** 2
# fit a spline to the Morse data on the Chebyshev grid
V_r_spline = interpolate.UnivariateSpline(r, V_r, k=5)
fit_error = V_r - V_r_spline(r)
fit_error_squared = fit_error ** 2
sum_squared_error = np.sum(fit_error_squared)
mse = sum_squared_error / N_points
print(F'MSE is {mse}')
# now define the expansion coefficients to expand the Morse potential as a polynomial
# harmonic
k = 2 * De_au * beta_au ** 2
f_spline = V_r_spline.derivative()
k_spline = f_spline.derivative()
# compare k_num to k
k_num = k_spline(r_eq_au)
print(k, k_num)
assert np.isclose(k, k_num, 1e-4) # this fails - it should pass!
#g_num = g_spline(r_eq_SI)
#h_num = h_spline(r_eq_SI)

In [ ]:
chebyfit = np.polynomial.Chebyshev.fit(r_array, cavity_free_E_array_CO[:,0], 49)
print(chebyfit)

cheby_array = np.zeros((1, N_R))
for i in range(N_R):
    cheby_array[0,i] = -7.83315327 + 0.01665548 * (r_array[i]) + 0.03414176 * (r_array[i]) ** 2 # - 0.02189794 * r_array[i] ** 3 + 0.00669785 * r_array[i] ** 4
cheby_list = np.ndarray.flatten(cheby_array)

print(cheby_array)
plt.plot(r_array, cavity_free_E_array_CO[:,0], 'ro')
plt.plot(r_array, cheby_list)
plt.show

## Calculation of the Huang-Rhys Factor
Huang Rhys factor can be calculated by both

$$ S = 1/2(\Delta x / x_0)^2 \tag{Turner}$$

from the mode anharmonicity paper

and 

$$ S = \frac{m\omega_{vib} \Delta x^2}{2 \hbar} \tag{Hsu}$$

from the polaritonic Huang-Rhys factor paper

In [ ]:
from scipy import constants
m_H = constants.m_p + constants.m_e 
print(m_H * constants.N_A * 1000)
m_Li = 3 * constants.m_p + 4 * constants.m_n + 3 * constants.m_e
# Constants and Variables
x_0 = wfn_CO.frequency_analysis["Xtp0"].data[5]
k_SI = wfn_CO.frequency_analysis["k"].data[5] * 10 ** 2
print(x_0)
min_S1_loc = np.argmin(cavity_free_E_array_CO[:,1])
min_S0_loc = np.argmin(cavity_free_E_array_CO[:,0])
print(f'Min on S0 is {r_array[min_S0_loc]}')
print(f'Min on S1 is {r_array[min_S1_loc]}')
delta_au = (r_array[min_S1_loc] - r_array[min_S0_loc]) / psi4.constants.bohr2angstroms
delta_m = (r_array[min_S1_loc] - r_array[min_S0_loc]) * 10 ** (-10)
mu_kg = (m_H * m_Li) / (m_H + m_Li)
omega_vib = np.sqrt(k_SI / mu_kg)
h_bar = constants.hbar
r_eq = r_array[min_S0_loc]

# Turner
S_Turner = 0.5 * (delta_au / x_0) ** 2

# Hsu
S_Hsu = mu_kg * omega_vib * delta_m ** 2 / (2 * h_bar)

print(S_Turner)
print(S_Hsu)

In [ ]:
D_e_Hartree = cavity_free_E_array_CO[49,0] - cavity_free_E_array_CO[min_S0_loc,0] 
D_e = 0.09848201074825358
D_e_J = D_e_Hartree * 4.3597 * 10 ** (-18)
alpha_inv_m = np.sqrt(k_SI / (2 * D_e_J))
alpha_inv_Angs = alpha_inv_m * 10 ** (-10)
kappa = 2 * D_e_Hartree * alpha_inv_Angs ** 2
g = -6 * D_e_Hartree * alpha_inv_Angs ** 3
h = 14 * D_e_Hartree * alpha_inv_Angs ** 4

In [ ]:
V_r = D_e_Hartree * (1 - np.exp(-alpha_inv_Angs * (r_array - r_eq))) ** 2

V_r_spline = interpolate.UnivariateSpline(r_array, V_r, k=4)

# now define the expansion coefficients to expand the Morse potential as a polynomial
# harmonic
f_spline = V_r_spline.derivative()
k_spline = f_spline.derivative()
g_spline = k_spline.derivative()
h_spline = g_spline.derivative()

k_num = k_spline(r_eq)
g_num = g_spline(r_eq)
h_num = h_spline(r_eq)



V_H = 1/2 * k_num * (r_array - r_eq) ** 2

# cubic
#g = -6 * D_e * alpha ** 3

V_C = V_H + 1/6 * g_num * (r_array - r_eq) ** 3

# quartic 
#h = 14 * D_e * alpha ** 4

V_Q = V_H + V_C + 1/24 * h_num * (r_array - r_eq) ** 4


plt.plot(r_array, V_r, 'red')
#plt.plot(r_array, V_r_spline(r), 'blue')
#plt.plot(r, V_C, 'green')
plt.plot(r_array, V_Q, 'purple')
#plt.xlim(0.75 * r_eq, 1.5 * r_eq)
#plt.ylim(0, 2e-18)
plt.show()
print(V_Q)

In [ ]:
def quartic_morse(r, De, k, re, g, h):
    return (1 / 2) * k * (r - re) ** 2 + (1 / 6) * g * (r - re) ** 3 + (1 / 24) * h * (r - re) ** 4
    #return De * (1 - np.exp(-k * (r - re))) ** 2 + g * (r - re) ** 3 + h * (r - re) ** 4

# Initial parameter guesses
initial_guess = [D_e, kappa, r_eq, g, h]

# Perform the curve fit
fit_params, covariance = curve_fit(quartic_morse, r_array, cavity_free_E_array_CO[:,0], p0=initial_guess, maxfev=10000)

# Extract the optimized parameters
De_fit, k_fit, re_fit, g_fit, h_fit = fit_params

V_fit = quartic_morse(r_array, De_fit, k_fit, re_fit, g_fit, h_fit)

print(V_fit)

First plot the ground-state potential energy surfaces for LiH inside and outisde the cavity.  The effect of the cavity will raise the energy slightly.

In [ ]:
plt.plot(r_array, cavity_free_E_array_CO[:,0], 'r-', label='cavity free LiH |g>')
#plt.plot(r_array, cavity_free_E_array_LiH[:,1], 'b-', label='cavity free LiH |e>')
plt.plot(r_array, V_fit, label='fit')
plt.legend()
plt.show()



Next plot the ground state surface shifted by the cavity energy and the first excited state surface of 
LiH outside the cavity along with the lower- and upper-polariton surfaces from LiH inside the cavity.
Note that lower polariton surface intersects with another surface for the molecule in the cavity (see the crossing of the red and green curves around r = 1.5 Angstroms); the polariton surface is smooth, so we should really re-sort the eigenvalues around this point. Ruby has some experience doing this. 

In [ ]:
plt.plot(r_array, cavity_free_E_array_CO[:,0]+0.12086, label='cavity free |g>+$\omega$')
plt.plot(r_array, cavity_free_E_array_CO[:,1], label='cavity free |e>')
plt.plot(r_array, cavity_E_array_CO[:,1], label='cavity |LP>')
plt.plot(r_array, cavity_E_array_CO[:,2], label='cavity |LP>')
plt.plot(r_array, cavity_E_array_CO[:,3], label='cavity |UP>')
plt.xlim(1.0, 2.2)
plt.ylim(-7.78, -7.63)
plt.legend()
plt.show()